<a href="https://colab.research.google.com/github/fjadidi2001/Cyber-Attack-Detection/blob/main/DL4cyber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setup Environment and Unzip the Dataset

- Install necessary libraries.
- Mount Google Drive and unzip the dataset.

# Step 2: Load and Preprocess Data

- Combine all CSV files into a single DataFrame.
- Handle missing values, encode categorical features, and normalize numerical features.
- Split into training, validation, and test sets.
- Visualize class distribution and feature correlations.

# Step 3: Build and Train the Deep Learning Model